# Importing libraries

In [17]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import service #used to manage the lifecycle of the ChromeDriver service.
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import requests
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
import csv
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np

# Initialize WebDriver

In [5]:
driver = webdriver.Chrome()

# Get Url

In [6]:
url = "https://www.swiggy.com/city/patna"

In [7]:
driver.get(url)

# Wait for the page to load

In [8]:
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "body")))

[<selenium.webdriver.remote.webelement.WebElement (session="b7c1d1e18deef877f2c1f927a2bc278b", element="f.B48D62E8328E1CA051A4510E208421A4.d.FA0D376E12B3BAE07E5461F34A4E0EC9.e.319")>]

# Find all restaurant elements

In [9]:
restaurants = driver.find_elements(By.CLASS_NAME, "sc-hhyLtv")

# Initialize empty list to store restaurant data

In [10]:
restaurant_data = []

In [11]:
for restaurant in restaurants:
    try:
        # Extract all elements with the shared class
        details = restaurant.find_elements(By.CLASS_NAME, "BEWkR")

        # Logic to differentiate Address & Cuisine
        address, cuisine = "N/A", "N/A"
        for detail in details:
            text = detail.text.strip()
            if "₹" in text:  # If it contains ₹, it's likely a price, ignore it
                continue
            if "," in text:  # Addresses usually have commas (e.g., "Boring Road, Patna")
                address = text
            else:  # Otherwise, assume it's Cuisine
                cuisine = text

        # Extract other details
        name = restaurant.find_element(By.CLASS_NAME, "hxenBp").text
        avg_price = restaurant.find_element(By.CLASS_NAME, "jCPgxj").text
        rating = restaurant.find_element(By.CLASS_NAME, "dnBtAy").text

        # Append data to the list
        restaurant_data.append({
            "Name": name,
            "Average Dish Price": avg_price,
            "Address": address,
            "Cuisine": cuisine,
            "Rating": rating,
        })

    except Exception as e:
        print(f"Error extracting restaurant data: {e}")
        continue

# Close the WebDriver

In [ ]:
driver.quit()

# Save data to CSV

In [13]:
df = pd.DataFrame(restaurant_data)
df.to_csv("swiggy_restaurants.csv", index=False, encoding="utf-8")

# Print the DataFrame

In [14]:
print(df)

                                   Name Average Dish Price  \
0       The Great Kabab Factory Express      ₹1900 for two   
1                        Cilantro India      ₹1200 for two   
2                      Paradise Biryani                      
3                 Skyline Biryani House       ₹500 for two   
4    Ghoomar Tradional Thali Restaurant      ₹1000 for two   
..                                  ...                ...   
290                       Chicago Pizza                      
291                    Young's Hub Cafe       ₹600 for two   
292                       Terrace Grill      ₹1000 for two   
293                         Food Xpress                      
294                          Rock House      ₹1300 for two   

                                 Address                      Cuisine Rating  
0    Brij Saroj Complex, Khajpura, Patna       Chinese • North Indian    3.9  
1                       Shekhpura, Patna        North Indian • Andhra    3.9  
2                 

In [3]:
df.count()

Name                  14
Average Dish Price    14
Address               14
Cuisine               14
Rating                14
dtype: int64

In [15]:
df

,Name,Average Dish Price,Address,Cuisine,Rating
0,The Great Kabab Factory Express,₹1900 for two,"Brij Saroj Complex, Khajpura, Patna",Chinese • North Indian,3.9
1,Cilantro India,₹1200 for two,"Shekhpura, Patna",North Indian • Andhra,3.9
2,Paradise Biryani,,"Khajpura, Patna",Chinese • North Indian,
3,Skyline Biryani House,₹500 for two,"Bailey Road, Patna",Biryani • North Indian,4.0
4,Ghoomar Tradional Thali Restaurant,₹1000 for two,"Bailey Road, Patna",North Indian • Rajasthani,4.7
...,...,...,...,...,...
290,Chicago Pizza,,"Ranipur, Patna",Italian • Fast Food,
291,Young's Hub Cafe,₹600 for two,"Marufganj, Patna",North Indian • Pasta,--
292,Terrace Grill,₹1000 for two,"Boring Canal Road, Patna",Italian • North Indian,--
293,Food Xpress,,"Adalatganj, Patna",North Indian • South Indian,


In [23]:
df.shape

(295, 5)

In [19]:
pd.isnull(df).sum()

Name                  0
Average Dish Price    0
Address               0
Cuisine               0
Rating                0
dtype: int64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295 entries, 0 to 294
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Name                295 non-null    object
 1   Average Dish Price  295 non-null    object
 2   Address             295 non-null    object
 3   Cuisine             295 non-null    object
 4   Rating              295 non-null    object
dtypes: object(5)
memory usage: 11.7+ KB


In [26]:
import mysql.connector

In [41]:
conn = mysql.connector.connect(host = "localhost",
                               user = "root",
                               password = "root",
                               database = "scraping")
                               

In [42]:
cursor = conn.cursor()

In [47]:
df = df.fillna({'Name': 'Unknown', 'Average Dish Price': '0', 'Address': 'Not Available', 'Cuisine': 'Unknown', 'Rating': 'N/A'})


In [40]:
# Ensure the database is selected
cursor.execute("USE scraping;")  

In [48]:
# Replace NaN with empty string
df = df.fillna('')  

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO swiggy (Name, Average_Dish_Price, Address, Cuisine, Rating)
        VALUES (%s, %s, %s, %s, %s)
    """, (row["Name"], row["Average Dish Price"], row["Address"], row["Cuisine"], row["Rating"]))



In [ ]:
 # Ensure changes are saved
conn.commit() 

In [52]:
querry = "SELECT * FROM Swiggy"
cursor.execute(querry)
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'The Great Kabab Factory Express', '₹1900 for two', 'Brij Saroj Complex, Khajpura, Patna', 'Chinese • North Indian', '3.9')
(2, 'Cilantro India', '₹1200 for two', 'Shekhpura, Patna', 'North Indian • Andhra', '3.9')
(3, 'The Great Kabab Factory Express', '₹1900 for two', 'Brij Saroj Complex, Khajpura, Patna', 'Chinese • North Indian', '3.9')
(4, 'Cilantro India', '₹1200 for two', 'Shekhpura, Patna', 'North Indian • Andhra', '3.9')
(5, 'Paradise Biryani', '', 'Khajpura, Patna', 'Chinese • North Indian', '')
(6, 'Skyline Biryani House', '₹500 for two', 'Bailey Road, Patna', 'Biryani • North Indian', '4')
(7, 'Ghoomar Tradional Thali Restaurant', '₹1000 for two', 'Bailey Road, Patna', 'North Indian • Rajasthani', '4.7')
(8, 'Le Cafe Crush', '', 'Khajpura, Patna', 'Chinese • North Indian', '')
(9, 'Kathi Nation', '₹600 for two', 'Khajpura, Patna', 'Mughlai • North Indian', '4.4')
(10, 'Wisteria Cafe & Kitchen', '₹1000 for two', 'Shekhpura, Patna', 'North Indian • Chinese', '4.5')
(11, '

In [53]:
cursor.close()

True

In [54]:
conn.close()